These are some libraries that will need to be downloaded if the next cell throws some ModuleNotFound errors. Delete the # of the line who is missing and run the cell.

In [ ]:
%%capture
!pip install pandas
!pip install google-cloud-storage

# **Library Imports and Back End Setup**
Nothing needs to be changed here, this cell just imports all the libraries our code needs to run, and gets Google Drive mounted to this notebook. This is also where our helper functions live so run this for sure!

You **will** need to approve this notebook to having access to your local Google Drive, it will be a little pop up when this cell gets run!

In [ ]:
import tempfile
import re
import pandas as pd
from io import FileIO
from google.cloud import storage
from google.oauth2 import service_account
from google.colab import drive

drive.mount('/content/gdrive')
_client = storage.Client.from_service_account_json("/content/gdrive/Shareddrives/LCLLC/fuelcast-storage-credentials.json")


def download_temp(bucket: str, remote_path: str) -> tempfile.NamedTemporaryFile:
      """[summary]
      Download a file from GCS and write it to a temporary file on disk. Return the named
      temporary file.
      Args:
      :param bucket_name: The name of the source bucket
      :param remote_path: The source path of the source blob.
      :return:
      """
      bucket = _client.bucket(bucket)
      blob = bucket.blob(remote_path)

      fp = tempfile.NamedTemporaryFile()

      _client.download_blob_to_file(blob, fp)
      fp.seek(0)
      return fp

def upload_temp(
        bucket_name: str, source_file_obj: FileIO, destination_blob_name: str
    ) -> None:
        """[summary]
        Upload a file object from disk. Works with temp files, should also work with "real" files as long
        as they're open as a file object. TODO test real files
        Args:
        :param bucket_name: Bucket to upload file to
        :param source_file_obj: An active file object to upload
        :param destination_blob_name: A name, including any "subdirectories", to upload the blob to (but not bucket)
        :return:
        """
        bucket = _client.bucket(bucket_name)
        blob = bucket.blob(destination_blob_name)

        blob.upload_from_file(source_file_obj, rewind=True)
        # We return the blob object in order to make the temporary file public for download in main.py
        return blob

def list_blobs_names(bucket_name: str, p: str = "") -> list:
    """[summary]
    Return a list of all blob names in the given bucket matching the optional prefix p
    Args:
    :param bucket_name: The name of the source bucket
    :param p: An optional destination folder path if the sourced files are in a specific folder.
    :return:
    """
    blobs = _client.list_blobs(bucket_name, prefix=p)
    blobnames = [x.name for x in blobs]
    return blobnames

# **Input** 

file_name: The name of the file you used when running the GEE script.

In [ ]:
file_name="test"

# **Data Compiler**
You don't need to change anything in here, just let the cell run.

In [ ]:

csvs = list_blobs_names("fuelcast-public","Matt/"+file_name)

def num_sort(test_string):
    return list(map(int, re.findall(r'\d+', test_string)))[0]
csvs.sort(key=num_sort)

csv_list = []
for i in csvs:
    csv_list.append(pd.read_csv(download_temp("fuelcast-public", i)))
result = pd.concat(csv_list)
print(result)
result.to_csv("/content/gdrive/MyDrive/"+file_name+".csv")
